In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import copy
from tqdm import tqdm

# 데이터 경로 설정
train_dir = './train'
test_dir = './test'
labels_file = './labels.csv'

# 라벨 -> 인덱스 변환 및 반대 변환
def create_label_mappings(labels_df):
    breed_to_idx = {breed: idx for idx, breed in enumerate(sorted(labels_df['breed'].unique()))}
    idx_to_breed = {idx: breed for breed, idx in breed_to_idx.items()}
    return breed_to_idx, idx_to_breed

# 데이터셋 전처리 변환 설정
def get_transforms(augment=False):
    if augment:
        return transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    else:
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

# Dataset 클래스 정의
class DogBreedDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, is_test=False):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0] + '.jpg')
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        if not self.is_test:
            breed_name = self.dataframe.iloc[idx, 1]
            label = breed_to_idx[breed_name]
            return image, label
        else:
            return image, self.dataframe.iloc[idx, 0]

# 데이터셋 및 데이터로더 생성 함수
def create_data_loaders(train_df, val_df, batch_size, img_dir):
    train_dataset = DogBreedDataset(dataframe=train_df, img_dir=img_dir, transform=get_transforms(augment=True))
    val_dataset = DogBreedDataset(dataframe=val_df, img_dir=img_dir, transform=get_transforms(augment=False))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

# 모델 학습 함수
def train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs, device, early_stop_limit=3, best_model_path='best_model.pth'):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    early_stop_count = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')

        # 학습 단계
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            running_loss += loss.item() * inputs.size(0)

        # 검증 단계
        model.eval()
        val_loss = 0.0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # 성능 지표 계산
        val_loss /= len(val_loader.dataset)
        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='weighted')

        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}')

        # 베스트 모델 저장
        if accuracy > best_acc:
            best_acc = accuracy
            best_model_wts = copy.deepcopy(model.state_dict())
            early_stop_count = 0
            torch.save(model.state_dict(), best_model_path)
            print(f'Best model saved at epoch {epoch+1} with validation accuracy: {accuracy:.4f}')
        else:
            early_stop_count += 1

        # 조기 종료 체크
        if early_stop_count >= early_stop_limit:
            print("Early stopping triggered")
            break

    model.load_state_dict(best_model_wts)
    return model, best_acc

# 테스트 데이터셋에 대한 예측
def predict_test(model, test_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = torch.softmax(outputs, dim=1)
            predictions.extend(probabilities.cpu().numpy())
    return predictions

# 제출 파일 생성 함수
def create_submission(predictions, idx_to_breed, test_df, output_file):
    predictions_df = pd.DataFrame(predictions, columns=idx_to_breed.values())
    predictions_df.insert(0, 'id', test_df['id'])
    predictions_df.to_csv(output_file, index=False)
    print(f"제출 파일 생성 완료: {output_file}")

# 메인 실행 부분
if __name__ == "__main__":
    # 레이블 파일 읽기
    labels_df = pd.read_csv(labels_file)
    
    # 라벨 -> 인덱스 변환 및 반대 변환
    breed_to_idx, idx_to_breed = create_label_mappings(labels_df)

    # 데이터셋 분리
    train_df, val_df = train_test_split(labels_df, test_size=0.2, stratify=labels_df['breed'], random_state=42)

    # 데이터 로더 생성
    train_loader, val_loader = create_data_loaders(train_df, val_df, batch_size=32, img_dir=train_dir)

    # 클래스 가중치 계산
    class_weights = compute_class_weight('balanced', classes=np.unique(labels_df['breed']), y=labels_df['breed'])
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    # 사전 훈련된 ResNet50 모델 로드 및 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = models.resnet50(weights='ResNet50_Weights.DEFAULT')
    model.fc = nn.Linear(model.fc.in_features, len(breed_to_idx))
    model = model.to(device)

    # 옵티마이저, 스케줄러 및 손실 함수 설정
    optimizer = optim.AdamW(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, steps_per_epoch=len(train_loader), epochs=20)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

    # 베스트 모델 파일 경로 설정
    best_model_path = 'best_model_5.pth'

    # 모델 학습
    model, best_acc = train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=20, device=device, best_model_path=best_model_path)
    print(f"Best Validation Accuracy: {best_acc:.4f}")

    # 최종 베스트 모델 로드 (저장된 파일에서)
    model.load_state_dict(torch.load(best_model_path))
    model = model.to(device)
    print(f'Best model loaded from {best_model_path}')

    # 테스트 데이터 예측 및 제출 파일 생성
    test_df = pd.DataFrame({'id': [f.split('.')[0] for f in os.listdir(test_dir)]})
    test_dataset = DogBreedDataset(dataframe=test_df, img_dir=test_dir, transform=get_transforms(augment=False), is_test=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    predictions = predict_test(model, test_loader, device)
    create_submission(predictions, idx_to_breed, test_df, output_file='submission_5.csv')


Epoch 1/20


100%|██████████| 256/256 [02:54<00:00,  1.47it/s]


Validation Loss: 1.4550, Accuracy: 0.7164, F1-Score: 0.6952
Best model saved at epoch 1 with validation accuracy: 0.7164
Epoch 2/20


100%|██████████| 256/256 [03:04<00:00,  1.39it/s]


Validation Loss: 1.3631, Accuracy: 0.6284, F1-Score: 0.6170
Epoch 3/20


...

Validation Loss: 1.5884, Accuracy: 0.5482, F1-Score: 0.5371
Epoch 4/20


100%|██████████| 256/256 [04:23<00:00,  1.03s/it]


Validation Loss: 1.9523, Accuracy: 0.4641, F1-Score: 0.4535
Early stopping triggered
Best Validation Accuracy: 0.7164
Best model loaded from best_model_5.pth


/tmp/ipykernel_27912/1351767038.py:197: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


제출 파일 생성 완료: submission_5.csv


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import copy

# 데이터 경로 설정
train_dir = './train'
test_dir = './test'
labels_file = './labels.csv'

# 라벨 -> 인덱스 변환 및 반대 변환
def create_label_mappings(labels_df):
    breed_to_idx = {breed: idx for idx, breed in enumerate(sorted(labels_df['breed'].unique()))}
    idx_to_breed = {idx: breed for breed, idx in breed_to_idx.items()}
    return breed_to_idx, idx_to_breed

# 데이터셋 전처리 변환 설정 (데이터 증강 포함)
def get_transforms(augment=False):
    if augment:
        return transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(30),  # 회전 범위를 30도로 확대
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            transforms.RandomGrayscale(p=0.2),  # 일부 이미지를 흑백으로 변환
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    else:
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

# Dataset 클래스 정의
class DogBreedDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, is_test=False):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0] + '.jpg')
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        if not self.is_test:
            breed_name = self.dataframe.iloc[idx, 1]
            label = breed_to_idx[breed_name]
            return image, label
        else:
            return image, self.dataframe.iloc[idx, 0]

# 데이터셋 및 데이터로더 생성 함수
def create_data_loaders(train_df, val_df, batch_size, img_dir):
    train_dataset = DogBreedDataset(dataframe=train_df, img_dir=img_dir, transform=get_transforms(augment=True))
    val_dataset = DogBreedDataset(dataframe=val_df, img_dir=img_dir, transform=get_transforms(augment=False))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

# 모델 정의 (드롭아웃 추가)
class ModifiedResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedResNet50, self).__init__()
        self.model = models.resnet50(weights='ResNet50_Weights.DEFAULT')
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Dropout(0.5),  # 드롭아웃 추가
            nn.Linear(num_ftrs, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# 모델 학습 함수
def train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs, device, early_stop_limit=3, best_model_path='best_model.pth'):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    early_stop_count = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')

        # 학습 단계
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            running_loss += loss.item() * inputs.size(0)

        # 검증 단계
        model.eval()
        val_loss = 0.0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # 성능 지표 계산
        val_loss /= len(val_loader.dataset)
        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='weighted')

        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}')

        # 베스트 모델 저장
        if accuracy > best_acc:
            best_acc = accuracy
            best_model_wts = copy.deepcopy(model.state_dict())
            early_stop_count = 0
            torch.save(model.state_dict(), best_model_path)
            print(f'Best model saved at epoch {epoch+1} with validation accuracy: {accuracy:.4f}')
        else:
            early_stop_count += 1

        # 조기 종료 체크
        if early_stop_count >= early_stop_limit:
            print("Early stopping triggered")
            break

    model.load_state_dict(best_model_wts)
    return model, best_acc

# 테스트 데이터셋에 대한 예측
def predict_test(model, test_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = torch.softmax(outputs, dim=1)
            predictions.extend(probabilities.cpu().numpy())
    return predictions

# 제출 파일 생성 함수
def create_submission(predictions, idx_to_breed, test_df, output_file='submission.csv'):
    predictions_df = pd.DataFrame(predictions, columns=idx_to_breed.values())
    predictions_df.insert(0, 'id', test_df['id'])
    predictions_df.to_csv(output_file, index=False)
    print(f"제출 파일 생성 완료: {output_file}")

# 메인 실행 부분
if __name__ == "__main__":
    # 레이블 파일 읽기
    labels_df = pd.read_csv(labels_file)
    
    # 라벨 -> 인덱스 변환 및 반대 변환
    breed_to_idx, idx_to_breed = create_label_mappings(labels_df)

    # 데이터셋 분리 (전체 데이터셋 사용)
    train_df, val_df = train_test_split(labels_df, test_size=0.2, stratify=labels_df['breed'], random_state=42)

    # 데이터 로더 생성
    train_loader, val_loader = create_data_loaders(train_df, val_df, batch_size=32, img_dir=train_dir)

    # 클래스 가중치 계산
    class_weights = compute_class_weight('balanced', classes=np.unique(labels_df['breed']), y=labels_df['breed'])
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    # 사전 훈련된 ResNet50 모델 로드 및 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ModifiedResNet50(num_classes=len(breed_to_idx))
    model = model.to(device)

    # 옵티마이저, 스케줄러 및 손실 함수 설정
    optimizer = optim.AdamW(model.parameters(), lr=1e-4)  # 학습률을 1e-4로 재조정
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-4, steps_per_epoch=len(train_loader), epochs=20)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

    # 베스트 모델 파일 경로 설정
    best_model_path = 'best_model.pth'

    # 모델 학습
    model, best_acc = train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=20, device=device, best_model_path=best_model_path)
    print(f"Best Validation Accuracy: {best_acc:.4f}")

    # 테스트 데이터 로드 및 예측
    test_df = pd.DataFrame({'id': [f.split('.')[0] for f in os.listdir(test_dir)]})
    test_dataset = DogBreedDataset(dataframe=test_df, img_dir=test_dir, transform=get_transforms(augment=False), is_test=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    predictions = predict_test(model, test_loader, device)

    # 제출 파일 생성
    create_submission(predictions, idx_to_breed, test_df)


Epoch 1/20
Validation Loss: 4.7443, Accuracy: 0.0323, F1-Score: 0.0206
Best model saved at epoch 1 with validation accuracy: 0.0323
Epoch 2/20
Validation Loss: 4.2662, Accuracy: 0.3320, F1-Score: 0.2986
Best model saved at epoch 2 with validation accuracy: 0.3320
Epoch 3/20
Validation Loss: 1.5744, Accuracy: 0.7178, F1-Score: 0.6953
Best model saved at epoch 3 with validation accuracy: 0.7178
Epoch 4/20
Validation Loss: 1.0081, Accuracy: 0.7467, F1-Score: 0.7365
Best model saved at epoch 4 with validation accuracy: 0.7467
Epoch 5/20
Validation Loss: 0.8008, Accuracy: 0.7702, F1-Score: 0.7609
Best model saved at epoch 5 with validation accuracy: 0.7702
Epoch 6/20
Validation Loss: 0.7651, Accuracy: 0.7619, F1-Score: 0.7536
Epoch 7/20
Validation Loss: 0.7050, Accuracy: 0.7775, F1-Score: 0.7716
Best model saved at epoch 7 with validation accuracy: 0.7775
Epoch 8/20
Validation Loss: 0.6890, Accuracy: 0.7956, F1-Score: 0.7926
Best model saved at epoch 8 with validation accuracy: 0.7956
Epoch

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
from torchvision.models import efficientnet_b0
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import copy

# 데이터 경로 설정
train_dir = './train'
test_dir = './test'
labels_file = './labels.csv'

# 라벨 -> 인덱스 변환 및 반대 변환
def create_label_mappings(labels_df):
    breed_to_idx = {breed: idx for idx, breed in enumerate(sorted(labels_df['breed'].unique()))}
    idx_to_breed = {idx: breed for breed, idx in breed_to_idx.items()}
    return breed_to_idx, idx_to_breed

# 데이터셋 전처리 변환 설정 (데이터 증강 제거)
def get_transforms():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

# Dataset 클래스 정의
class DogBreedDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, is_test=False):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0] + '.jpg')
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        if not self.is_test:
            breed_name = self.dataframe.iloc[idx, 1]
            label = breed_to_idx[breed_name]
            return image, label
        else:
            return image, self.dataframe.iloc[idx, 0]

# 데이터셋 및 데이터로더 생성 함수
def create_data_loaders(train_df, val_df, batch_size, img_dir):
    train_dataset = DogBreedDataset(dataframe=train_df, img_dir=img_dir, transform=get_transforms())
    val_dataset = DogBreedDataset(dataframe=val_df, img_dir=img_dir, transform=get_transforms())

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

# EfficientNet 모델 정의
class EfficientNetModel(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetModel, self).__init__()
        self.model = efficientnet_b0(pretrained=True)  # EfficientNet-B0 사용
        num_ftrs = self.model.classifier[1].in_features
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(num_ftrs, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# 모델 학습 함수
def train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs, device, early_stop_limit=3, best_model_path='best_model.pth'):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    early_stop_count = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')

        # 학습 단계
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        # 검증 단계
        model.eval()
        val_loss = 0.0
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # 성능 지표 계산
        val_loss /= len(val_loader.dataset)
        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='weighted')

        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}')

        # 베스트 모델 저장
        if accuracy > best_acc:
            best_acc = accuracy
            best_model_wts = copy.deepcopy(model.state_dict())
            early_stop_count = 0
            torch.save(model.state_dict(), best_model_path)
            print(f'Best model saved at epoch {epoch+1} with validation accuracy: {accuracy:.4f}')
        else:
            early_stop_count += 1

        # 스케줄러 업데이트
        scheduler.step()

        # 조기 종료 체크
        if early_stop_count >= early_stop_limit:
            print("Early stopping triggered")
            break

    model.load_state_dict(best_model_wts)
    return model, best_acc

# 테스트 데이터셋에 대한 예측
def predict_test(model, test_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = torch.softmax(outputs, dim=1)
            predictions.extend(probabilities.cpu().numpy())
    return predictions

# 제출 파일 생성 함수
def create_submission(predictions, idx_to_breed, test_df, output_file='submission.csv'):
    predictions_df = pd.DataFrame(predictions, columns=idx_to_breed.values())
    predictions_df.insert(0, 'id', test_df['id'])
    predictions_df.to_csv(output_file, index=False)
    print(f"제출 파일 생성 완료: {output_file}")

# 메인 실행 부분
if __name__ == "__main__":
    # 레이블 파일 읽기
    labels_df = pd.read_csv(labels_file)
    
    # 라벨 -> 인덱스 변환 및 반대 변환
    breed_to_idx, idx_to_breed = create_label_mappings(labels_df)

    # 데이터셋 분리 (전체 데이터셋 사용)
    train_df, val_df = train_test_split(labels_df, test_size=0.2, stratify=labels_df['breed'], random_state=42)

    # 데이터 로더 생성
    train_loader, val_loader = create_data_loaders(train_df, val_df, batch_size=32, img_dir=train_dir)

    # 클래스 가중치 계산
    class_weights = compute_class_weight('balanced', classes=np.unique(labels_df['breed']), y=labels_df['breed'])
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

    # EfficientNet 모델 로드 및 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = EfficientNetModel(num_classes=len(breed_to_idx))
    model = model.to(device)

    # 옵티마이저, 스케줄러 및 손실 함수 설정
    optimizer = optim.AdamW(model.parameters(), lr=3e-4)  # EfficientNet에 맞춘 학습률 설정
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1)  # 스케줄러 적용
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

    # 베스트 모델 파일 경로 설정
    best_model_path = 'best_efficientnet_model.pth'

    # 모델 학습
    model, best_acc = train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=20, device=device, best_model_path=best_model_path)

    print(f'Best Validation Accuracy: {best_acc:.4f}')

    # 테스트 데이터 로드 및 예측
    test_df = pd.DataFrame({'id': [f.split('.')[0] for f in os.listdir(test_dir)]})
    test_dataset = DogBreedDataset(dataframe=test_df, img_dir=test_dir, transform=get_transforms(), is_test=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    predictions = predict_test(model, test_loader, device)

    # 제출 파일 생성
    create_submission(predictions, idx_to_breed, test_df)


/home/sdf1ai810/miniconda3/envs/module01/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sdf1ai810/miniconda3/envs/module01/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/20
Validation Loss: 1.1568, Accuracy: 0.6851, F1-Score: 0.6730
Best model saved at epoch 1 with validation accuracy: 0.6851
Epoch 2/20
Validation Loss: 0.9097, Accuracy: 0.7252, F1-Score: 0.7192
Best model saved at epoch 2 with validation accuracy: 0.7252
Epoch 3/20
Validation Loss: 0.8307, Accuracy: 0.7443, F1-Score: 0.7387
Best model saved at epoch 3 with validation accuracy: 0.7443
Epoch 4/20
Validation Loss: 0.8329, Accuracy: 0.7457, F1-Score: 0.7432
Best model saved at epoch 4 with validation accuracy: 0.7457
Epoch 5/20
Validation Loss: 0.8282, Accuracy: 0.7565, F1-Score: 0.7535
Best model saved at epoch 5 with validation accuracy: 0.7565
Epoch 6/20
Validation Loss: 0.8334, Accuracy: 0.7560, F1-Score: 0.7528
Epoch 7/20
Validation Loss: 0.8083, Accuracy: 0.7667, F1-Score: 0.7645
Best model saved at epoch 7 with validation accuracy: 0.7667
Epoch 8/20
Validation Loss: 0.8210, Accuracy: 0.7697, F1-Score: 0.7659
Best model saved at epoch 8 with validation accuracy: 0.7697
Epoch